In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')

In [2]:
initial_Features=pd.read_csv('tox21_global_cdf_rdkit.csv')
initial_dataset=pd.read_csv('tox21.csv')
initial_Features=initial_Features.loc[:,initial_Features.apply(pd.Series.nunique) != 1]
initial_dataset=initial_dataset.iloc[initial_Features.dropna().index]
initial_dataset=initial_dataset.reset_index()
initial_Features=initial_Features.dropna()
initial_Features=initial_Features.reset_index()
index_array=[]
for i in np.arange(1,13):
    index_array.append(initial_dataset.iloc[:,i+1].dropna().index)

In [3]:
def label_ith(i):
    return pd.DataFrame(data=initial_dataset.iloc[index_array[i]].iloc[:,i+2])
def Feature_ith(i):
    return initial_Features.iloc[index_array[i]].drop('index',axis=1)

In [4]:
from sklearn.model_selection import train_test_split
X_training_data=[]
X_test=[]
y_training_data=[]
y_test=[]
for i in np.arange(0,12):
    X_training_data_tmp, X_test_tmp, y_training_data_tmp, y_test_tmp =train_test_split(Feature_ith(i),label_ith(i), stratify=label_ith(i),test_size=0.10,random_state=1234)
    X_training_data.append(X_training_data_tmp)
    X_test.append(X_test_tmp)
    y_training_data.append(y_training_data_tmp)
    y_test.append(y_test_tmp)

from sklearn.decomposition import PCA
X_training_data_pca=[]
X_test_pca=[]
for i in np.arange(0,12):
    pca = PCA(n_components=71)
    principalComponents = pca.fit_transform(X_training_data[i])
    X_training_data_PCA_tmp = pd.DataFrame(data = principalComponents)
    X_training_data_pca.append(X_training_data_PCA_tmp)
    X_test_pca_tmp=pd.DataFrame(data=pca.transform(X_test[i]))
    X_test_pca.append(X_test_pca_tmp)
    print('PCA with 71 principal components retains',np.sum(pca.explained_variance_ratio_)*100,'% of data VAR. (It is related for',i,'th label)')

PCA with 71 principal components retains 95.14902477458284 % of data VAR. (It is related for 0 th label)
PCA with 71 principal components retains 95.18528509261213 % of data VAR. (It is related for 1 th label)
PCA with 71 principal components retains 95.23516716185081 % of data VAR. (It is related for 2 th label)
PCA with 71 principal components retains 95.24899205532745 % of data VAR. (It is related for 3 th label)
PCA with 71 principal components retains 95.19568949095981 % of data VAR. (It is related for 4 th label)
PCA with 71 principal components retains 95.18825061528679 % of data VAR. (It is related for 5 th label)
PCA with 71 principal components retains 95.13894464393015 % of data VAR. (It is related for 6 th label)
PCA with 71 principal components retains 95.21618305641543 % of data VAR. (It is related for 7 th label)
PCA with 71 principal components retains 95.18018989506741 % of data VAR. (It is related for 8 th label)
PCA with 71 principal components retains 95.16545227328

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix,plot_confusion_matrix

In [7]:
print("KNeighborsClassifier:")
for i in np.arange(0,12,1):
    from sklearn.neighbors import KNeighborsClassifier
    knn = KNeighborsClassifier()
    param_grid={'n_neighbors':np.arange(1,11,2),'metric':['euclidean','cosine','manhattan']}
    grid=GridSearchCV(knn,param_grid=param_grid,cv=10,scoring='balanced_accuracy',return_train_score=False)
    grid.fit(X_training_data_pca[i].to_numpy(),y_training_data[i])
    print("-----------------------This information is for the ",i,"th label.-----------------------")
    print("-------------------------------------Including PCA.-------------------------------------")
    print("best mean cv score= ",grid.best_score_)
    print("best parameters= ",grid.best_params_)
    y_pred_knn_pca=grid.best_estimator_.predict(X_test_pca[i])
    print(confusion_matrix(y_test[i],np.transpose(np.matrix(y_pred_knn_pca))))
    print(grid.score(X_test_pca[i],y_test[i]))

KNeighborsClassifier:
-----------------------This information is for the  0 th label.-----------------------
-------------------------------------Including PCA.-------------------------------------
best mean cv score=  0.7235540265998088
best parameters=  {'metric': 'manhattan', 'n_neighbors': 7}
[[682   4]
 [ 15  16]]
0.7551490642339885
-----------------------This information is for the  1 th label.-----------------------
-------------------------------------Including PCA.-------------------------------------
best mean cv score=  0.7850023245692129
best parameters=  {'metric': 'manhattan', 'n_neighbors': 1}
[[637   9]
 [  6  17]]
0.8625992731188585
-----------------------This information is for the  2 th label.-----------------------
-------------------------------------Including PCA.-------------------------------------
best mean cv score=  0.7259460692370788
best parameters=  {'metric': 'cosine', 'n_neighbors': 1}
[[544  27]
 [ 33  44]]
0.7620715536652489
-----------------------This

In [8]:
print("KNeighborsClassifier:")
for i in np.arange(0,12,1):
    knn = KNeighborsClassifier()
    param_grid={'n_neighbors':np.arange(1,11,2),'metric':['euclidean','cosine','manhattan']}
    grid=GridSearchCV(knn,param_grid=param_grid,cv=10,scoring='balanced_accuracy',return_train_score=False)
    grid.fit(X_training_data[i].to_numpy(),y_training_data[i])
    print("-----------------------This information is for the ",i,"th label.-----------------------")
    #print("-------------------------------------Including PCA.-------------------------------------")
    print("best mean cv score= ",grid.best_score_)
    print("best parameters= ",grid.best_params_)
    y_pred_knn=grid.best_estimator_.predict(X_test[i])
    print(confusion_matrix(y_test[i],np.transpose(np.matrix(y_pred_knn))))
    print(grid.score(X_test[i],y_test[i]))

KNeighborsClassifier:
-----------------------This information is for the  0 th label.-----------------------
best mean cv score=  0.7309100650238083
best parameters=  {'metric': 'manhattan', 'n_neighbors': 9}
[[683   3]
 [ 15  16]]
0.7558779272077494
-----------------------This information is for the  1 th label.-----------------------
best mean cv score=  0.7846450333353117
best parameters=  {'metric': 'manhattan', 'n_neighbors': 1}
[[634  12]
 [  6  17]]
0.8602772916947099
-----------------------This information is for the  2 th label.-----------------------
best mean cv score=  0.7272648019793841
best parameters=  {'metric': 'manhattan', 'n_neighbors': 1}
[[545  26]
 [ 32  45]]
0.7694407169013123
-----------------------This information is for the  3 th label.-----------------------
best mean cv score=  0.6781417541315211
best parameters=  {'metric': 'manhattan', 'n_neighbors': 1}
[[528  19]
 [ 13  17]]
0.7659658744667885
-----------------------This information is for the  4 th label

In [9]:
print("DecisionTreeClassifier:")
from sklearn.tree import DecisionTreeClassifier, export_graphviz
for i in np.arange(0,12,1):
    DecisionTree = DecisionTreeClassifier()
    param_grid={'criterion':['gini','entropy'],'max_depth':np.arange(4,30,1)}
    grid=GridSearchCV(DecisionTree,param_grid=param_grid,cv=10,scoring='balanced_accuracy',return_train_score=False)
    grid.fit(X_training_data[i].to_numpy(),y_training_data[i])
    print("-----------------------This information is for the ",i,"th label.----------------------------")
    print("best mean cv score= ",grid.best_score_)
    print("best parameters= ",grid.best_params_)
    y_pred_dt=grid.best_estimator_.predict(X_test[i])
    print(confusion_matrix(y_test[i],np.transpose(np.matrix(y_pred_dt))))
    print(grid.score(X_test[i],y_test[i]))

DecisionTreeClassifier:
-----------------------This information is for the  0 th label.----------------------------
best mean cv score=  0.7343343187519519
best parameters=  {'criterion': 'entropy', 'max_depth': 4}
[[682   4]
 [ 15  16]]
0.7551490642339885
-----------------------This information is for the  1 th label.----------------------------
best mean cv score=  0.7493516924842225
best parameters=  {'criterion': 'gini', 'max_depth': 27}
[[637   9]
 [  7  16]]
0.8408601426840758
-----------------------This information is for the  2 th label.----------------------------
best mean cv score=  0.715454431723965
best parameters=  {'criterion': 'gini', 'max_depth': 27}
[[534  37]
 [ 34  43]]
0.7468214797461732
-----------------------This information is for the  3 th label.----------------------------
best mean cv score=  0.6357916597143412
best parameters=  {'criterion': 'gini', 'max_depth': 22}
[[523  24]
 [ 19  11]]
0.6613954905545399
-----------------------This information is for the 

In [11]:
print("RandomForestClassifier:")
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
for i in np.arange(0,12,1):
    RFC = RandomForestClassifier()
    param_grid={'n_estimators':[500,700],'criterion':['gini','entropy'],'max_depth':[4,6,8,10,15,20], 'max_features':['auto','sqrt','log2']}
    grid=GridSearchCV(RFC,param_grid=param_grid,cv=5,scoring='balanced_accuracy',return_train_score=False)
    grid.fit(X_training_data[i].to_numpy(),y_training_data[i])
    print("-----------------------This information is for the ",i,"th label.----------------------------")
    print("best mean cv score= ",grid.best_score_)
    print("best parameters= ",grid.best_params_)
    y_pred_RFC=grid.best_estimator_.predict(X_test[i])
    print(confusion_matrix(y_test[i],np.transpose(np.matrix(y_pred_RFC))))
    print(grid.score(X_test[i],y_test[i]))

RandomForestClassifier:
-----------------------This information is for the  0 th label.----------------------------
best mean cv score=  0.7216258089347315
best parameters=  {'criterion': 'gini', 'max_depth': 15, 'max_features': 'auto', 'n_estimators': 700}
[[684   2]
 [ 15  16]]
0.7566067901815103
-----------------------This information is for the  1 th label.----------------------------
best mean cv score=  0.739999752677349
best parameters=  {'criterion': 'entropy', 'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 500}
[[640   6]
 [  6  17]]
0.8649212545430072
-----------------------This information is for the  2 th label.----------------------------
best mean cv score=  0.6722003183198237
best parameters=  {'criterion': 'entropy', 'max_depth': 20, 'max_features': 'auto', 'n_estimators': 700}
[[560  11]
 [ 46  31]]
0.6916664771305752
-----------------------This information is for the  3 th label.----------------------------
best mean cv score=  0.5665454586815841
best parame

In [13]:
from xgboost import XGBClassifier
print("XGBClassifier:")
for i in np.arange(0,12,1):
    param_grid={'n_estimators':[700,2000],'max_depth':[3,4,5,6,7],'eval_metric':['mlogloss']}
    grid=GridSearchCV(XGBClassifier(),param_grid=param_grid,cv=10,scoring='balanced_accuracy',return_train_score=False)
    grid.fit(X_training_data[i].to_numpy(),y_training_data[i])
    print("-----------------------This information is for the ",i,"th label.----------------------------")
    print("best mean cv score= ",grid.best_score_)
    print("best parameters= ",grid.best_params_)
    y_pred_xgbc=grid.best_estimator_.predict(X_test[i])
    print(confusion_matrix(y_test[i],np.transpose(np.matrix(y_pred_xgbc))))
    print(grid.score(X_test[i],y_test[i]))

XGBClassifier:
-----------------------This information is for the  0 th label.----------------------------
best mean cv score=  0.7220923302944042
best parameters=  {'eval_metric': 'mlogloss', 'max_depth': 3, 'n_estimators': 700}
[[683   3]
 [ 16  15]]
0.7397488949496849
-----------------------This information is for the  1 th label.----------------------------
best mean cv score=  0.7588862345935466
best parameters=  {'eval_metric': 'mlogloss', 'max_depth': 6, 'n_estimators': 2000}
[[640   6]
 [  8  15]]
0.821442993673442
-----------------------This information is for the  2 th label.----------------------------
best mean cv score=  0.7333850964654818
best parameters=  {'eval_metric': 'mlogloss', 'max_depth': 5, 'n_estimators': 2000}
[[557  14]
 [ 36  41]]
0.7539745718379693
-----------------------This information is for the  3 th label.----------------------------
best mean cv score=  0.6181325689242583
best parameters=  {'eval_metric': 'mlogloss', 'max_depth': 5, 'n_estimators': 700

In [14]:
from sklearn.svm import SVC
print("SVC:")
for i in np.arange(0,12,1):
    param_grid={'kernel':['poly'],'degree':np.arange(1,15,1)}
    grid=GridSearchCV(SVC(),param_grid=param_grid,cv=10,scoring='balanced_accuracy',return_train_score=False)
    grid.fit(X_training_data[i].to_numpy(),y_training_data[i])
    print("-----------------------This information is for the ",i,"th label.----------------------------")
    print("best mean cv score= ",grid.best_score_)
    print("best parameters= ",grid.best_params_)
    y_pred_svm=grid.best_estimator_.predict(X_test[i])
    print(confusion_matrix(y_test[i],np.transpose(np.matrix(y_pred_svm))))
    print(grid.score(X_test[i],y_test[i]))

XGBClassifier:
-----------------------This information is for the  0 th label.----------------------------
best mean cv score=  0.7230681963217643
best parameters=  {'degree': 4, 'kernel': 'poly'}
[[680   6]
 [ 15  16]]
0.7536913382864667
-----------------------This information is for the  1 th label.----------------------------
best mean cv score=  0.7546105110095553
best parameters=  {'degree': 5, 'kernel': 'poly'}
[[638   8]
 [  8  15]]
0.8198950060573429
-----------------------This information is for the  2 th label.----------------------------
best mean cv score=  0.7242156849858815
best parameters=  {'degree': 5, 'kernel': 'poly'}
[[546  25]
 [ 38  39]]
0.7313553346828303
-----------------------This information is for the  3 th label.----------------------------
best mean cv score=  0.6645036425232391
best parameters=  {'degree': 7, 'kernel': 'poly'}
[[534  13]
 [ 17  13]]
0.7047836684948202
-----------------------This information is for the  4 th label.--------------------------

In [15]:
from sklearn.ensemble import AdaBoostClassifier
print("AdaBoostClassifier:")
for i in np.arange(0,12,1):
    param_grid={'n_estimators':[8000,1000,500,700],'algorithm':['SAMME.R','SAMME'],'learning_rate':[1]}
    grid=GridSearchCV(AdaBoostClassifier(),param_grid=param_grid,cv=5,scoring='balanced_accuracy',return_train_score=False)
    grid.fit(X_training_data[i].to_numpy(),y_training_data[i])
    print("-----------------------This information is for the ",i,"th label.----------------------------")
    print("best mean cv score= ",grid.best_score_)
    print("best parameters= ",grid.best_params_)
    y_pred_adaboost=grid.best_estimator_.predict(X_test[i])
    print(confusion_matrix(y_test[i],np.transpose(np.matrix(y_pred_adaboost))))
    print(grid.score(X_test[i],y_test[i]))

AdaBoostClassifier:
-----------------------This information is for the  0 th label.----------------------------
best mean cv score=  0.7157613477540118
best parameters=  {'algorithm': 'SAMME.R', 'learning_rate': 1, 'n_estimators': 8000}
[[674  12]
 [ 16  15]]
0.7331891281858365
-----------------------This information is for the  1 th label.----------------------------
best mean cv score=  0.7341166085078775
best parameters=  {'algorithm': 'SAMME.R', 'learning_rate': 1, 'n_estimators': 500}
[[640   6]
 [  6  17]]
0.8649212545430072
-----------------------This information is for the  2 th label.----------------------------
best mean cv score=  0.7308967630172937
best parameters=  {'algorithm': 'SAMME.R', 'learning_rate': 1, 'n_estimators': 8000}
[[535  36]
 [ 35  42]]
0.7412036299952236
-----------------------This information is for the  3 th label.----------------------------
best mean cv score=  0.6154114251647196
best parameters=  {'algorithm': 'SAMME.R', 'learning_rate': 1, 'n_estima

In [16]:
from sklearn.ensemble import BaggingClassifier
print("BaggingClassifier:")
for i in np.arange(0,12,1):
    param_grid={'n_estimators':[10,100,500],'max_samples':[1,0.5],'max_features':[1,0.8,0.6]}
    grid=GridSearchCV(BaggingClassifier(),param_grid=param_grid,cv=5,scoring='balanced_accuracy',return_train_score=False)
    grid.fit(X_training_data[i].to_numpy(),y_training_data[i])
    print("-----------------------This information is for the ",i,"th label.----------------------------")
    print("best mean cv score= ",grid.best_score_)
    print("best parameters= ",grid.best_params_)
    y_pred_bag=grid.best_estimator_.predict(X_test[i])
    print(confusion_matrix(y_test[i],np.transpose(np.matrix(y_pred_bag))))
    print(grid.score(X_test[i],y_test[i]))

AdaBoostClassifier:
-----------------------This information is for the  0 th label.----------------------------
best mean cv score=  0.7269508786019065
best parameters=  {'max_features': 0.6, 'max_samples': 0.5, 'n_estimators': 500}
[[683   3]
 [ 15  16]]
0.7558779272077494
-----------------------This information is for the  1 th label.----------------------------
best mean cv score=  0.7326826795066174
best parameters=  {'max_features': 0.8, 'max_samples': 0.5, 'n_estimators': 100}
[[640   6]
 [  8  15]]
0.821442993673442
-----------------------This information is for the  2 th label.----------------------------
best mean cv score=  0.6652733608745625
best parameters=  {'max_features': 0.6, 'max_samples': 0.5, 'n_estimators': 100}
[[560  11]
 [ 48  29]]
0.6786794641435623
-----------------------This information is for the  3 th label.----------------------------
best mean cv score=  0.559236688270835
best parameters=  {'max_features': 0.8, 'max_samples': 0.5, 'n_estimators': 500}
[[54